<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex14noteC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex14noteC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

---
## 密度推定と統計的パターン認識の実験
---

----
### 正規分布モデルではうまくいかない例

noteA,Bでは正規分布モデルによる密度推定がうまくいく例ばかり見せていました．うまくいかない例を観察するのも大事ですから，ここでそういう例を紹介しておきます．


まずはデータの準備．

In [ ]:
from scipy.stats import multivariate_normal
from sklearn.datasets import make_moons

moonX, moonY = make_moons(n_samples=200, noise=0.1)
XX = moonX[moonY==0, :]
print(XX.shape)

2次元のデータ100個です．このデータに2次元の正規分布を当てはめてみると...

In [ ]:
# 一つの正規分布を当てはめてみる
mu = np.mean(XX, axis=0)
cov = (XX - mu).T @ (XX - mu) / XX.shape[0]

# グラフ描画用のグリッドデータの作成
xmin, xmax = -1.5, 1.5
ymin, ymax = -1, 2
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))

# グラフ
fig = plt.figure(facecolor="white", figsize=(12, 8))

# 散布図
ax0 = fig.add_subplot(121)
ax0.scatter(XX[:, 0], XX[:, 1])
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# Gaussian を当てはめた結果
ax1 = fig.add_subplot(122)
ax1.scatter(XX[:, 0], XX[:, 1])
ax1.contour(x_mesh, y_mesh, multivariate_normal.pdf(X_mesh, mean=mu, cov=cov))
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

左の散布図を見て分かるように，このデータは正規分布とはまったく異なる分布の仕方をしています．これに正規分布を当てはめると，右図のような結果となります．データの分布を全く再現できてませんね．当てはめた正規分布の平均の近くは確率密度の値が大きいはずですが，その辺りには全くデータが存在していません．



というわけですので，例えば以下のような2クラスの識別問題に対して...

In [ ]:
# グラフを描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.scatter(moonX[moonY==0, 0], moonX[moonY==0, 1])
ax.scatter(moonX[moonY==1, 0], moonX[moonY==1, 1])
ax.set_xlim(-1.5, 2.5)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
plt.show()

それぞれのクラスのデータの分布に正規分布を当てはめて事後確率を推定してみると...

In [ ]:
K = 2
X = moonX
lab = moonY
N, D = X.shape

# 事前確率を推定
py = np.empty(K)
for k in range(K):
    py[k] = np.sum(lab==k)/N
print('### 事前確率')
print('p(y) = ', py)

# p(x|y) を正規分布でモデル化（平均，共分散行列の推定）
mu = np.empty((K, D))
cov = np.empty((K, D, D))
for k in range(K):
    XX = X[lab==k, :]
    mu[k] = np.mean(XX, axis=0)
    cov[k] = (XX - mu[k]).T @ (XX - mu[k]) / XX.shape[0]
    print(f'### class {k} の正規分布のパラメータ')
    print(mu[k])
    print(cov[k])

# グラフ描画用のグリッドデータの作成
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))

# 事後確率の推定
p = np.empty((K, X_mesh.shape[0]*X_mesh.shape[1]))
for k in range(K):
    p[k, :] = py[k] * multivariate_normal.pdf(X_mesh, mean=mu[k], cov=cov[k]).reshape(-1)
p /= np.sum(p, axis=0)
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(12, 8))

# Gaussian を当てはめた結果
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(X[lab==k, 0], X[lab==k, 1])
    ax0.contour(x_mesh, y_mesh, multivariate_normal.pdf(X_mesh, mean=mu[k], cov=cov[k]))
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 事後確率の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(X[lab==k, 0], X[lab==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

こんなふうに．学習データすらまともに識別できない結果となっています．

というわけで，密度推定においては，データの分布に合わせた確率モデルを選ばないと良い結果は得られません．これは，回帰の問題で入力と出力の関係が直線の式ではうまく表せないようなデータに直線を当てはめようとしているのと似た状況ですね．

※ 「**よだんだよん**」のところにこの話の続きの余談があります．

----
### 手書き数字識別やってみる

クラスごとのデータの分布を正規分布をと仮定して事後確率を推定する方法で，手書き数字識別やってみましょう．


#### 準備

In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
datL = rv['datL'].astype(float)
labL = rv['labL']
datT = rv['datT'].astype(float)
labT = rv['labT']
print(datL.shape, labL.shape, datT.shape, labT.shape)

K = 10 # クラス数
D = datL.shape[1] # データの次元数 28 x 28 = 784

In [ ]:
# データを画像として表示するための関数
#
def display(data, nx, ny, nrow=28, ncol=28, gap=4):

    assert data.shape[0] == nx*ny
    assert data.shape[1] == nrow*ncol

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = data[iy*nx+ix].reshape((nrow, ncol))

    # 画像の出力
    plt.axis('off')
    plt.imshow(img, cmap = 'gray')
    plt.show()

In [ ]:
nx, ny = 10, 5
display(datL[:50], nx, ny)
for iy in range(ny):
    print(labL[iy*nx:(iy+1)*nx])

In [ ]:
XL = datL
YL = labL
NL, D = datL.shape
K = 10

#### 事前確率を推定

クラスごとの学習データの比率でクラス事前確率 $p(y)$ を推定．

In [ ]:
py = np.empty(K)
for k in range(K):
    py[k] = np.sum(YL==k)/NL
print('### 事前確率')
print('p(y) = ', py)

#### 正規分布の当てはめ

クラスごとのデータの分布 $p(\mathbf{x}|y)$ を推定．
多変量正規分布でモデル化し，それぞれの平均と分散共分散行列を求めます．

In [ ]:
from sklearn.mixture import GaussianMixture

model = np.empty(K, dtype=object)
for k in range(K):
    XX = XL[YL==k, :]
    model[k] = GaussianMixture(n_components=1, covariance_type='full')
    model[k].fit(XX)

クラスごとのデータの平均を可視化してみるとこんなん．

In [ ]:
xm = np.empty((K, D))
for k in range(K):
    xm[k, :] = model[k].means_[0, :]
display(xm, K, 1)

#### テストデータを識別させてみる

テストデータを事後確率の推定値が最大となるクラスへ分類する方法で，識別率を算出してみます．
クラスごとのデータの平均をプロトタイプとする最短距離法でも同じことをやり，結果を比較します．

In [ ]:
XT = datT
YT = labT
NT, _ = datT.shape

score = np.empty(K)
ncorrect1 = ncorrect2 = 0

for n in range(NT):
    xx = XT[n, :]

    # 方法1: 事後確率の推定値が最大のクラスに識別
    for k in range(K):
        # log(p(y)p(x|y)) = log(p(y)) + log(p(x|y))
        score[k] = np.log(py[k]) + model[k].score(xx[np.newaxis, :])
    if np.argmax(score) == YT[n]:
        ncorrect1 += 1

    # 方法2: 最短距離法で識別
    d2 = np.sum(np.square(xx[np.newaxis, :] - xm), axis=1)
    if np.argmin(d2) == YT[n]:
        ncorrect2 += 1

print(f'方法1の識別率: {ncorrect1}/{NT} = {ncorrect1/NT:.3f}')
print(f'方法2の識別率: {ncorrect2}/{NT} = {ncorrect2/NT:.3f}')

最短距離法に劣る結果となりました．
正規分布の当てはめがあまりうまくいっていないのかもしれません．

※ 「**よだんだよん**」のところにこの話の続きの余談があります．

---
### ［よだんだよん］ 混合正規分布モデルを使ってみる

※ ここから先は，授業の範囲を超えた余談です

上記で見たように，正規分布は単純すぎてうまくデータの分布を表現できないことがあります．
そういう場合への対応の方法はいろいろありますが，ここでは，「複数の正規分布の重み付き和」で表される「混合正規分布モデル」(Gaussian mixture model)を紹介します．

これは，$M$個の正規分布の重み付き和で，次のように表されます．
$$
p(\mathbf{x}) = \sum_{m=1}^{M} w_m{\cal N}(\mathbf{x}|\mathbf{\mu}_m, \Sigma_m)
$$
${\cal N}(\mathbf{x}|\mathbf{\mu}_m, \Sigma_m)$ は平均$\mathbf{\mu}_m$ 分散共分散行列 $\Sigma_m$ の多変量正規分布です．$w_m$ は $m$ 番目の正規分布の重みを表し，$0 \leq w_m \leq 1, \sum_{m=1}^M w_m = 1$ を満たします．
混合正規分布モデルでは，$w_m, \mathbf{\mu}_m, \Sigma_m$ がモデルパラメータとなります．

2次元の「うまくいかない例」のデータに $M=3$ の混合正規分布モデルを当てはめてみると，こんなふうになります．

In [ ]:
K = 2
X = moonX
lab = moonY
N, D = X.shape
M = 3

# 事前確率を推定
py = np.empty(K)
for k in range(K):
    py[k] = np.sum(lab==k)/N
print('### 事前確率')
print('p(y) = ', py)

# p(x|y) を多変量正規分布でモデル化
mu = np.empty((K, M, D))
cov = np.empty((K, M, D, D))
model = np.empty(K, dtype=object)
for k in range(K):
    XX = X[lab==k, :]
    model[k] = GaussianMixture(n_components=M, covariance_type='full')
    model[k].fit(XX)
    mu[k, ::] = model[k].means_
    cov[k, ::] = model[k].covariances_
    

# グラフ描画用のグリッドデータの作成
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))
print(X_mesh.shape)

# 事後確率の推定
p = np.empty((K, X_mesh.shape[0]*X_mesh.shape[1]))
for k in range(K):
    proba = np.exp(model[k].score_samples(X_mesh.reshape((-1, 2))))
    p[k, :] = py[k] * proba
p /= np.sum(p, axis=0)
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(12, 8))

# Gaussian を当てはめた結果
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(X[lab==k, 0], X[lab==k, 1])
    for m in range(M):
        z = multivariate_normal.pdf(X_mesh, mean=mu[k, m], cov=cov[k, m])
        ax0.contour(x_mesh, y_mesh, z, levels=6)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 事後確率の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(X[lab==k, 0], X[lab==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

次に，$M = 5$ の混合正規分布モデルを手書き数字識別に使ってみます．

まずは，クラスごとのデータに混合正規分布モデルを当てはめます．

In [ ]:
from sklearn.mixture import GaussianMixture

XL = datL
YL = labL
NL, D = datL.shape
K = 10
M = 5

model = np.empty(K, dtype=object)
for k in range(K):
    print()
    print(f'##### K = {k}')
    XX = XL[labL==k, :]
    model[k] = GaussianMixture(n_components=M, covariance_type='full', verbose=10)
    model[k].fit(XX)

クラスごとに$M$個の正規分布を使っています．それぞれの平均を可視化するとこんなんなります．

In [ ]:
mu = np.empty((K, M, D))
for k in range(K):
    mu[k, ::] = model[k].means_

display(mu.reshape((-1, D)), M, K)


識別させてみると...

In [ ]:
XT = datT
YT = labT
NT, _ = datT.shape

score = np.empty(K)
ncorrect1 = 0

# 事前確率を推定
py = np.empty(K)
for k in range(K):
    py[k] = np.sum(YL==k)/N
#print('### 事前確率')
#print('p(y) = ', py)

# 事後確率の推定値が最大のクラスに識別
for n in range(NT):
    xx = XT[n, :]
    for k in range(K):
        # log(p(y)p(x|y)) = log(p(y)) + log(p(x|y))
        score[k] = np.log(py[k]) + model[k].score(xx[np.newaxis, :])
    if np.argmax(score) == YT[n]:
        ncorrect1 += 1
    if n % 100 == 0:
        print(f'{ncorrect1}/{n}')

print(f'識別率: {ncorrect1}/{NT} = {ncorrect1/NT:.3f}')

正規分布一つや最短距離法よりも高い識別率が得られました．